In [ ]:
import geemap
import ee
from datetime import datetime

In [ ]:
cloud_project = 'ee-karanjanaphtali9'

try:
    ee.Initialize(project=cloud_project)
except:
    ee.Authenticate()
    ee.Initialize(project=cloud_project)

In [ ]:
# Create a map
m = geemap.Map()

# Define the area of interest (Bandipur National Park example; replace with your region)
geometry = ee.Geometry.Polygon([
    [38.44375658227224, 0.8472122850918858],
    [39.29519701195974, 0.8472122850918858],
    [39.29519701195974, 1.3579802593499926],
    [38.44375658227224, 1.3579802593499926],
    [38.44375658227224, 0.8472122850918858]
])

# Update dates for the ongoing fire
fireStart = ee.Date('2025-01-17')
fireEnd = ee.Date(datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%S'))  # Get current date

# Center map to the area of interest
m.centerObject(geometry, 10)

# Load Sentinel-2 image collection
s2 = ee.ImageCollection("COPERNICUS/S2")

# Apply filters to Sentinel-2 collection
filtered = s2.filter(ee.Filter.bounds(geometry)).select('B.*')

# Load the Cloud Score+ collection
csPlus = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')
csPlusBands = csPlus.first().bandNames()

# Add Cloud Score+ bands to Sentinel-2 images
filteredS2WithCs = filtered.map(lambda img: img.addBands(csPlus.filter(ee.Filter.eq('system:index', img.get('system:index'))).first()))

# Function to mask low-quality pixels based on Cloud Score+
def maskLowQA(image):
    qaBand = 'cs'
    clearThreshold = 0.5
    mask = image.select(qaBand).gte(clearThreshold)
    return image.updateMask(mask)

# Apply the mask to the filtered collection
filteredMasked = filteredS2WithCs.map(maskLowQA)

# Create Before and After composites
before = filteredMasked.filter(ee.Filter.date(fireStart.advance(-2, 'month'), fireStart)).median()
after = filteredMasked.filter(ee.Filter.date(fireStart, fireEnd)).median()

# False Color Visualization for SWIR bands
swirVis = {"min": 0.0, "max": 3000, "bands": ['B12', 'B8', 'B4']}
m.addLayer(before.clip(geometry), swirVis, 'Before')
m.addLayer(after.clip(geometry), swirVis, 'After')

# Function to calculate Normalized Burn Ratio (NBR)
def addNBR(image):
    nbr = image.normalizedDifference(['B8', 'B12']).rename(['nbr'])
    return image.addBands(nbr)

# Calculate NBR for Before and After composites
beforeNbr = addNBR(before).select('nbr')
afterNbr = addNBR(after).select('nbr')

# Visualization for NBR
nbrVis = {"min": -0.5, "max": 0.5, "palette": ['white', 'black']}
m.addLayer(beforeNbr.clip(geometry), nbrVis, 'Prefire NBR')
m.addLayer(afterNbr.clip(geometry), nbrVis, 'Postfire NBR')

# Calculate Change in NBR (dNBR)
change = beforeNbr.subtract(afterNbr)

# Apply a threshold to detect burned areas
threshold = 0.3
burned = change.gt(threshold)

# Display burned areas
m.addLayer(burned.clip(geometry), {"min": 0, "max": 1, "palette": ['white', 'red']}, 'Burned Areas', False)

# Display the map
m


Map(center=[1.102619397441078, 38.86947679711605], controls=(WidgetControl(options=['position', 'transparent_b…